# Estudo de Naive RAG
Naive RAG study

_Estas instruções foram retiradas do link: <https://towardsdatascience.com/retrieval-augmented-generation-rag-from-theory-to-langchain-implementation-4e9bd5f6a4f2>_


The above link was used to follow the instructions.

## Instalação de Pacotes
Package installation

- **Observação:** Pode ser necessário instalar novos pacotes ao longo das instruções
- **Note:** It may be necessary to install new packages throughout the instructions

In [1]:
# pip install langchain openai weaviate-client tiktoken python-dotenv pdfplumber
# pip install -U langchain-community

## Imports gerais
General Imports

In [2]:
import os
from dotenv import load_dotenv

## Carregar variáveis de ambiente
Load environment variables

In [3]:
load_dotenv()

# Chave API do OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Usando pdfplumber para extrair o texto que servirá como contexto para a IA
Using pdfplumber to extract the text that will serve as context for the AI

In [ ]:
import pdfplumber

with pdfplumber.open('./your_file.pdf') as pdf:
    text = ''
    for page in pdf.pages:
        text += page.extract_text()

In [ ]:
# Salvar o conteúdo em um arquivo / Save content into a file
with open('pdf_file_text.txt', 'w', encoding='utf-8') as file:
    file.write(text)

## Carregar contexto
Load context

- Este é o contexto que será fornecido à **LLM**
- This is the context that will be provided to the **LLM**

In [15]:
import requests
from langchain.document_loaders import TextLoader

# Usar contexto a partir de uma URL // Context from URL
url = "https://gist.githubusercontent.com/fzliu/973bb1d659a740b1d78a659f90be4a02/raw/6ff89ece130119011951f450164006221acadd8a/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()

## Segmentar o documento de texto (criar _chunks_)
Create chunks

- É necessário segmentar o texto em pedaços menores porque seu conteúdo é muito longo para caber na janela de contexto da **LLM**
- Será usada a lib **LangChain** para isso

- It is necessary to segment the text into smaller chunks because its content is too long to fit in the **LLM** context window
- The **LangChain** library will be used for this

In [16]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

## Incorporar (_embed_) e armazenar os segmentos
Embed and store chunks

- Para haver busca semântica pelos segmentos, é preciso gerar as incorporações em forma de vetor (_vector embeddings_) para cada segmento e armazená-las junto com suas incorporações
- Será usada a **OpenAI** para gerar os vetores incorporados e o **Weaviate vector db** para armazenar os vetores

- For semantic search of the segments, it is necessary to generate vector embeddings for each segment and store them along with their embeddings
- OpenAI will be used to generate the vector embeddings and Weaviate vector db to store the vectors

In [17]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
    embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

## **Passo 1:** _Retrieve_ (Recuperar)

- Uma vez que a base de dados vetorizada (_vector database_) está populada, é possível defini-la como o **componente de recuperação**, que busca um contexto adicional baseado na similaridade semântica entre a _query_ do usuário e os segmentos incorporados

In [18]:
retriever = vectorstore.as_retriever()

## **Passo 2:** _Augment_ (Aprimorar)

- Para aprimorar o prompt com o contexto adicional, é preciso preparar um **template do prompt**

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Você é um assistente para tarefas de perguntas e respostas.
Utilize os seguintes fragmentos de contexto recuperado para responder à pergunta.
Se você não souber a resposta, simplesmente diga que não sabe.
Não poupe palavras ou tokens na resposta.
Pergunta: {question}
Contexto: {context}
Resposta:
"""

prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Você é um assistente para tarefas de perguntas e respostas.\nUtilize os seguintes fragmentos de contexto recuperado para responder à pergunta.\nSe você não souber a resposta, simplesmente diga que não sabe.\nNão poupe palavras ou tokens na resposta.\nPergunta: {question}\nContexto: {context}\nResposta:\n'), additional_kwargs={})]


## **Passo 3:** _Generate_ (Gerar)

- Agora é possível gerar uma cadeia para o pipeline RAG, juntando o **retriever**, o **prompt** e a **LLM**
- Quando a cadeia RAG estiver definida, é possível invocá-la

- Now it is possible to generate a chain for the RAG pipeline, combining the **retriever**, the **prompt**, and the **LLM**
- Once the RAG chain is defined, it can be invoked

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=16000)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = """Me explique, em português, qual é o ponto principal do discurso."""

output = rag_chain.invoke(query)

/Users/romul/Docs/github_ia/rag_basico/env/lib/python3.11/site-packages/pydantic/main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/romul/Docs/github_ia/rag_basico/env/lib/python3.11/site-packages/pydantic/main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [23]:
print(f"Pergunta: {query}")
print(f"Resposta: {output}")

with open('output.md', 'w', encoding='utf-8') as file:
    file.write(output)

Pergunta: Me explique, em português, qual é o ponto principal do discurso.
Resposta: O ponto principal do discurso é a afirmação da força e resiliência da nação americana diante das adversidades. O orador expressa otimismo em relação ao futuro dos Estados Unidos, destacando que, apesar dos desafios enfrentados, o caráter e a determinação do povo americano são fundamentais para superar crises e transformar dificuldades em oportunidades. O discurso enfatiza a importância da liberdade, da justiça e da responsabilidade coletiva, além de reafirmar a posição dos Estados Unidos como uma nação que sempre busca expandir a democracia e a equidade. O orador também faz referência ao apoio à Ucrânia em sua luta pela liberdade, simbolizando a luta global contra a opressão. Em suma, o discurso é um chamado à ação e à unidade, ressaltando que a força da nação reside em seu povo.
